In [152]:
import pennylane as qml
from pennylane import numpy as np
# import autograd.numpy as np

import matplotlib.pyplot as plt


# == Test for two coefficient calculation if it's diff? ==
def With_norm(theta, gamma):
    coeff = np.array([ ((1+np.exp(-1.j * theta))/2) * (np.sqrt(1-gamma)) , (1-np.exp(-1.j * theta))/2 ]) / (1-gamma * (np.cos(theta)**2) )
    norm = np.linalg.norm(coeff)
    
    return coeff / norm
    # return round_oper(coeff_result.numpy())

def Without_norm(theta):
    gamma = 0
    coeff = np.array([ ((1+np.exp(-1.j * theta))/2) * (np.sqrt(1-gamma)) , (1-np.exp(-1.j * theta))/2 ]) / (1-gamma * (np.cos(theta)**2) )
    norm = 1
    
    return coeff / norm
    # return round_oper(coeff.numpy())

In [153]:
# precision
def round_oper(coeff_in, n):
    coeff_round = np.zeros(2, dtype='complex_')
    
    for i in range(2):
        coeff_round[i] = round(   qml.math.real(coeff_in[i])  , n) + round(   qml.math.imag(coeff_in[i])   , n) * 1j
    
    return coeff_round

# precision
def round_oper_1(coeff_in, n):
    coeff_round = np.zeros(2, dtype='complex_')
    
    for i in range(2):
        coeff_round[i] = round(   qml.math.real(coeff_in[i])  , n) + round(   qml.math.imag(coeff_in[i])   , n) * 1j
    
    return coeff_round

In [154]:
round_oper(With_norm(2.0,0).numpy(), 6)

tensor([0.291927-0.454649j, 0.708073+0.454649j], requires_grad=True)

In [155]:
# == Define new circuit for test ==
dev_without = qml.device('default.qubit', wires = 1)
@qml.qnode(dev_without)
def circuit_without(theta):
    qml.QubitStateVector(Without_norm(theta), wires=range(1))
    # qml.QubitStateVector(Without_norm(theta), wires=range(1))

    
    return qml.probs()
    # return qml.density_matrix(wires=0)


dev_with_norm = qml.device('default.qubit', wires = 1)
@qml.qnode(dev_with_norm)
def circuit_with_norm(theta):
    qml.QubitStateVector(round_oper(With_norm(theta,0).numpy(),6), wires=range(1))

    
    return qml.probs()
    # return qml.density_matrix(wires=0)
    
circuit_without(np.pi/2)


tensor([0.5, 0.5], requires_grad=True)

In [156]:
# == Compare with CFI plot ==

N = 1000
tau_CFI = np.linspace(-0.001, 3.0, N)

CFI_without = np.zeros(N)
CFI_with = np.zeros(N)

for i in range(len(tau_CFI)):
    CFI_without[i] = qml.qinfo.classical_fisher(circuit_without)(tau_CFI[i])
    CFI_with[i] = qml.qinfo.classical_fisher((circuit_with_norm))(tau_CFI[i])


print('== print out CFI ==')
plt.subplot(211)
plt.plot(tau_CFI, CFI_without)
plt.title('Without normalized')
plt.xlabel('Time')
plt.ylabel('Probability_0_state')
plt.grid()


plt.subplot(212)
plt.plot(tau_CFI, CFI_with)
plt.title('With normalized')
plt.xlabel('Time')
plt.ylabel('Probability_0_state')
# plt.legend()
plt.grid()

AttributeError: 'ArrayBox' object has no attribute 'numpy'